In [1]:
import pymongo
from pymongo import MongoClient
client = MongoClient()
client = MongoClient('localhost', 27017)
# client = MongoClient('mongodb://localhost:27017/')
db = client.db_cdiscount

In [2]:
import io
import bson                       # this is installed with the pymongo package
import matplotlib.pyplot as plt
from skimage.data import imread   # or, whatever image library you prefer
import multiprocessing as mp      # will come in handy due to the size of the data

In [3]:
import numpy as np

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
import keras
K.set_image_dim_ordering('th')

Using TensorFlow backend.


In [5]:
first = 0
last = 7069896
batch = 4000
num_batch = int(last/batch)
# last = 82


In [6]:
epochs = 4
batch_size = 16
num_classes = db.cat_encode.count()

In [7]:
def simple_model(epochs=epochs, num_classes=num_classes, lrate=0.01, momentum=0.9):
    
    # Convolutional Layer
    model = Sequential()
    # Convolutional Layer
    model.add(Conv2D(40, (3,3), input_shape = (180,180,3), activation='relu'))

    # Pooling Layer
    model.add(MaxPooling2D(pool_size=(2, 1)))
    
    # Fully conected Layer
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))


    lrate = 0.01
    decay = lrate/epochs
    sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)

#     model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [8]:
def deeper_model(epochs=epochs, num_classes=num_classes, lrate=0.01, momentum=0.9):
    # Convolutional Layer
    model = Sequential()
    # Convolutional Layer
    model.add(Conv2D(10, (3,3), input_shape = (180,180,3), activation='relu'))
    model.add(Conv2D(13, 3, activation='relu', padding='same'))
    model.add(Conv2D(14, 3, activation='relu', padding='same'))

    # Pooling Layer
    model.add(MaxPooling2D(pool_size=(2, 1)))
    
    
    model.add(Conv2D(20, 3, activation='relu', padding='same'))    
    model.add(Conv2D(26, 3, activation='relu', padding='same'))
    
    # Fully conected Layer
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))


    lrate = 0.01
    decay = lrate/epochs
    sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)

#     model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
    

In [9]:
# model =simple_model(epochs=epochs, num_classes=num_classes)
model =deeper_model(epochs=epochs, num_classes=num_classes)


In [10]:
import time

In [11]:
start_time = time.time()

counter = 0
row_count = 0
for j in range(0,num_batch-1):
# for j in range(0,1):
    
    a1 = int(list(np.linspace(first,last,num_batch))[j])
    an = int(list(np.linspace(first,last,num_batch))[j+1])
    
    lst_batch = []
    
#     print(a1,an)
    
    cur = db.train.find({})[a1:an]
    
    m= an-a1
        
    i = 0
#     pic_array
    while (cur.alive):
        idx = cur.next()
        dic = {}
        
        category_id =idx['category_id'] 
    #     print(category_id)

        cat = db.cat_encode.find_one({ "cat" : (category_id)}, {"cat" : 1.0, "_id" : 0})['cat']
    #     print(cat)
        
        picture = np.float32(imread(io.BytesIO(idx['imgs'][0]['picture']))/255.0)
        picture = picture.reshape(180,180,3)
        
        dic['picture'] = picture
        
        encode = db.cat_encode.find_one({ "cat" : (category_id)}, {"encode" : 1.0, "_id" : 0})['encode']
    #     print(encode)
        dic['encode'] = encode
        
        lst_batch.append(dic)
        
        i+=1
    X_batch = np.array([lst['picture'] for lst in lst_batch])
    Y_batch = np.array([lst['encode'] for lst in lst_batch])
    print('X batch size: ', X_batch.shape)
    print('Y batch size: ', Y_batch.shape)
    
    del(lst_batch)
        
    model.fit(X_batch, Y_batch, batch_size=batch_size, epochs=epochs, validation_split=0.2)
    
#     print(X_batch.shape[0])
    row_count = row_count + X_batch.shape[0]
    
    if(row_count >= 100000):
        print('row count: ',row_count)
        counter = counter + row_count
        model_name = 'model_' + str(counter) +'.h5'
        print(model_name)
        model.model.save('E://kaggle//Cdiscount//model//'+model_name)
        print('Model saved')
        row_count = 0
    
print("--- %s seconds ---" % (time.time() - start_time))
model.model.save('E://kaggle//Cdiscount//model//final_model.h5')

X batch size:  (600, 180, 180, 3)
Y batch size:  (600, 5270)
Train on 480 samples, validate on 120 samples
Epoch 1/4
480/480 [==============================] - 13s - loss: 6.2560 - acc: 0.3354 - val_loss: 5.1754 - val_acc: 0.4167
Epoch 2/4
480/480 [==============================] - 6s - loss: 3.8659 - acc: 0.4250 - val_loss: 4.5896 - val_acc: 0.4167
Epoch 3/4
480/480 [==============================] - 6s - loss: 3.5648 - acc: 0.4250 - val_loss: 4.8792 - val_acc: 0.4167
Epoch 4/4
480/480 [==============================] - 6s - loss: 3.4298 - acc: 0.4250 - val_loss: 4.6881 - val_acc: 0.4167
row count:  600
model_600.h5
Model saved
X batch size:  (600, 180, 180, 3)
Y batch size:  (600, 5270)
Train on 480 samples, validate on 120 samples
Epoch 1/4
480/480 [==============================] - 6s - loss: 4.4762 - acc: 0.4104 - val_loss: 3.6155 - val_acc: 0.4583
Epoch 2/4
480/480 [==============================] - 6s - loss: 3.5414 - acc: 0.4104 - val_loss: 3.6729 - val_acc: 0.4583
Epoch 3/4
48

In [12]:
# start_time = time.time()

# counter = 0
# for j in range(0,num_batch-1):
# # for j in range(0,1):
    
#     a1 = int(list(np.linspace(first,last,num_batch))[j])
#     an = int(list(np.linspace(first,last,num_batch))[j+1])
    
#     lst_batch = []
    
# #     print(a1,an)
    
#     cur = db.train.find({})[a1:an]
    
#     m= an-a1
        
#     i = 0
# #     pic_array
#     while (cur.alive):
#         idx = cur.next()
#         dic = {}
        
#         category_id =idx['category_id'] 
#     #     print(category_id)

#         cat = db.cat_encode.find_one({ "cat" : (category_id)}, {"cat" : 1.0, "_id" : 0})['cat']
#     #     print(cat)
        
#         picture = np.float32(imread(io.BytesIO(idx['imgs'][0]['picture']))/255.0)
#         picture = picture.reshape(180,180,3)
        
#         dic['picture'] = picture
        
#         encode = db.cat_encode.find_one({ "cat" : (category_id)}, {"encode" : 1.0, "_id" : 0})['encode']
#     #     print(encode)
#         dic['encode'] = encode
        
#         lst_batch.append(dic)
        
#         i+=1
#     X_batch = np.array([lst['picture'] for lst in lst_batch])
#     Y_batch = np.array([lst['encode'] for lst in lst_batch])
#     print(X_batch.shape)
#     print(Y_batch.shape)
    
#     del(lst_batch)
    
#     model.fit(X_batch, Y_batch, batch_size=batch_size, epochs=epochs, validation_split=0.2)
    
# print("--- %s seconds ---" % (time.time() - start_time))